In [2]:
import numpy as np
from pyvis.network import Network
import networkx as nx
from networkx import drawing as dw
import matplotlib.pyplot as plt
%matplotlib qt5

In [3]:
REAL_PATH = r"D:\code\studie\eds-quicksilver-21-team-4\workload\real\1\graph.nt"
SYN_PATH = r"D:\code\studie\eds-quicksilver-21-team-4\workload\syn\1\graph.nt"
LAYOUT = lambda g: dw.layout.spring_layout(g, seed=42, iterations=60)
# LAYOUT = lambda g: dw.layout.spectral_layout(g)
# LAYOUT = lambda g: dw.layout.planar_layout(g)

# Graph Initializers 


In [4]:
binary_generator = np.random.default_rng(seed=42)
def init():
    global C, N, S, G, count
    count = 0
    C = np.zeros((10,10), dtype='int')
    # N = np.random.randint(2, size=(10,10), )
    N = np.array([
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]])
    S = N.copy()
    G = nx.DiGraph(S)

def init2(size = 10):
    global C, N, S, G, count
    count = 0
    C = np.zeros((size,size), dtype='int')
    N = np.random.randint(2, size=(size,size), ) & np.random.randint(2, size=(size,size), )
    S = N.copy()
    G = nx.DiGraph(S)

def init15example():
    global C, N, S, G, count
    count = 0
    C = np.zeros((15,15), dtype='int')
    N = np.array([[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1],
       [1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1],
       [1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1],
       [1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0]])
    S = N.copy()
    G = nx.DiGraph(S)

def reset_N_and_C():
    global C, N, S, G, count
    count = 0
    C[:] = 0
    N = S.copy()
    S = N.copy()
    G = nx.DiGraph(S)

def initempty(size = 10):
    global C, N, S, G, count
    count = 0
    C = np.zeros((size,size), dtype='int')
    N = np.zeros((size,size), dtype='int')
    S = N.copy()
    G = nx.DiGraph(S)


def init_from_nx(graph, p=1):
    global C, N, S, G, count
    count = 0
    N = nx.to_numpy_array(graph, dtype='int')
    if 0 <= p < 1:
        mask = binary_generator.binomial(1, p, size=N.shape)
        N &= mask
    elif not p == 1: raise ValueError("Probability p must be in range [0, 1].")
    C = np.zeros_like(N)
    S = N.copy()
    G = nx.DiGraph(S)
    # G = graph

def init_TC_union(tc1, tc2, overlap=.5):
    assert tc1.shape == tc2.shape
    global C, N, S, G, count
    count = 0
    old_size = tc1.shape[0]
    new_v = int(old_size * (1-overlap))
    size = old_size + new_v
    C = np.zeros((size,size), dtype='int')
    C[:old_size,:old_size] = tc1
    N = np.zeros((size,size), dtype='int')
    N[new_v:,new_v:] = tc2
    S = C | N
    G = nx.DiGraph(S)
    

init()

In [5]:
def init_balanced_example():
    """Used in example video.
    
    Generated with:
        V = 20
        graph = nx.soft_random_geometric_graph(V, .85, 4)
        init_from_nx(graph, p=.35)
        drawnext(base_only=True)
    """
    global C, N, S, G, count
    count = 0
    N = np.array(
   [[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
    [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
    [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]])
    C = np.zeros_like(N)
    S = N.copy()
    G = nx.DiGraph(S)
init_balanced_example()

# Closure algorithm

In [6]:
def explore_debug(s, root):
    drawnext(s, color='red')
    for t in N[s].nonzero()[0]:
        if not C[root,t]: # needs to be checked as many times as N has ones.
            C[root, t] = 1 # after exploring, this node known by the root.
            if t == root: continue # Do not explore the root twice!
            drawnext(t, color='red')
            explore(t, root)
            # Backpropagation:
            C[s] |= C[t] # pull target's cover into current node
            C[s,t] = 1 # and add the target itself to the cover
            N[s] = ~C[s] & (N[s] | N[t])
            # drawnext(s, color='blue')
    drawnext(s, color='lime')
    
    overlap = (C[s] & N[s])
    assert not overlap.any(), f"{s=} overlap: {np.transpose(overlap.nonzero())}"

In [7]:
# Same algo, but no visualizations and invariant checks:
def explore(s, root):
    for t in N[s].nonzero()[0]:
        if not C[root,t]: # needs to be checked as many times as N has ones.
            C[root, t] = 1 # after exploring, this node known by the root.
            if t == root:
                N[root,t] = 0
                continue # Do not explore the root twice!
            explore(t, root)
            # Backpropagation:
            C[s] |= C[t] # pull target's cover into current node
            C[s,t] = 1 # and add the target itself to the cover
            N[s] = ~C[s] & (N[s] | N[t])


    overlap = (C[s] & N[s])
    assert not overlap.any(), f"{s=} overlap: {np.transpose(overlap.nonzero())}"

## Define Visualization Function

In [8]:
fig = plt.figure(1, figsize=(10,10))
plt.show()
plt.tight_layout()

def drawnext(s=None, color='lightgray', filter_c=None, base_only=False):
    global G, count

    plt.clf()
    
    layout = LAYOUT(G)
    # nx.draw_circular(G, with_labels=True, font_weight='bold', edge_color='blue', width=10, alpha=.3, )
    nx.draw_networkx_edges(
        G,
        layout,
        edge_color='blue',
        width=2 if filter_c else 3,
        alpha=.7 if filter_c else .5,
        node_size=400,
        arrowsize= 15 if filter_c else 20,
        min_source_margin=14,
        min_target_margin= 13 if filter_c else 5
    )
    nx.draw_networkx_nodes(G, layout, node_color='w', edgecolors='k', node_size=400)
    nx.draw_networkx_labels(G, layout)
    if base_only: return

    Gn = nx.DiGraph(N)
    Gc = nx.DiGraph(C)
    nx.draw_networkx_edges(Gn,
                           layout,
                           edge_color='red',
                           width=3.5,
                           alpha=.1 if filter_c else .7,
                           arrowstyle='-[',
                           min_target_margin=27,
                           arrowsize=12,
                           connectionstyle='arc3,rad=0.15')
    nx.draw_networkx_edges(Gc,
                           layout,
                           edge_color='k',
                           alpha=.7,
                           width=4 if filter_c else 3,
                           arrowsize=25 if filter_c else 15,
                           connectionstyle='arc3,rad=0.1',
                           edgelist=Gc.edges(filter_c))
    if s is not None or filter_c is not None:
        highlight_node = s if s is not None else filter_c
        select_color = color
        nx.draw_networkx_nodes(G,
                               layout,
                               node_color=select_color,
                               edgecolors='k',
                               node_size=1000,
                               nodelist=[highlight_node])
    # count+=1
    # plt.title(str(count))
    plt.pause(.2)
drawnext(1)
plt.tight_layout()

# Experiments 


## Solve Partial:


### Initialization


In [25]:
LAYOUT = lambda g: dw.layout.spring_layout(g, seed=42, iterations=30)
# LAYOUT = lambda g: dw.layout.spectral_layout(g)
# LAYOUT = lambda g: dw.layout.planar_layout(g)
drawnext(base_only=True)
plt.tight_layout()


In [8]:
V = 20
# graph = nx.path_graph(50, create_using=nx.DiGraph)
# graph = nx.to_directed(nx.watts_strogatz_graph(20,3, .4))
# graph = nx.random_k_out_graph(V, 3, 3)
# graph = nx.binomial_tree(5)
graph = nx.soft_random_geometric_graph(V, .85, 4)
init_from_nx(graph, p=.35)
drawnext(base_only=True)

In [49]:
init_balanced_example()
drawnext(base_only=True)

### Solve for single root

In [52]:
start = 1
drawnext(s=start)
explore(start, start)
drawnext()
overlap = (C & N)
if overlap.any():
    print("Overlap at:")
    print(np.transpose(overlap.nonzero()))
plt.pause(1)
drawnext(filter_c=start, color='lime')

### Small example

In [56]:
def shell_example(n=20, p=.01, start=19, keep_prob=1):
    graph = nx.to_directed(nx.watts_strogatz_graph(n,5, p, seed=10))
    init_from_nx(graph, keep_prob)
    LAYOUT = lambda g: dw.layout.spring_layout(g, seed=42, iterations=100)

def demonstrate(start=0):
    drawnext(s=start)
    plt.pause(5)
    explore(start, start)
    drawnext()
    overlap = (C & N)
    if overlap.any():
        print("Overlap at:")
        print(np.transpose(overlap.nonzero()))
    plt.pause(.1)
    drawnext(filter_c=start, color='lime')

In [54]:
shell_example(40, .6, start=1, keep_prob=.4)


In [58]:
demonstrate(4)

In [ ]:
reset_N_and_C()
drawnext(base_only=1)

## Test Complete Solve Time

In [9]:
import tqdm.notebook
def solve_kleene(show_progress=False):
    global G
    if show_progress:
        for root in tqdm.notebook.trange(G.number_of_nodes()):
            explore(root, root)
    else:
        for root in range(G.number_of_nodes()):
            explore(root, root)

In [40]:
%prun solve_kleene()


         36 function calls (28 primitive calls) in 0.000 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     15/7    0.000    0.000    0.000    0.000 3822328612.py:1(explore)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
       15    0.000    0.000    0.000    0.000 {method 'nonzero' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 3277688792.py:2(solve_kleene)
        1    0.000    0.000    0.000    0.000 graph.py:759(number_of_nodes)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [41]:
graph = nx.to_directed(nx.watts_strogatz_graph(30, 3, .45))
init_from_nx(graph, p=.8)
solve_kleene()
T1 = C.copy()
S1 = S.copy()

graph = nx.to_directed(nx.watts_strogatz_graph(30, 3, .4))
init_from_nx(graph, p=.9)
solve_kleene()
T2 = C.copy()
S2 = S.copy()

In [43]:
init_TC_union(T1, T2, overlap=0.1)
%prun solve_kleene()



         248 function calls (184 primitive calls) in 0.002 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   121/57    0.002    0.000    0.002    0.000 3822328612.py:1(explore)
      121    0.000    0.000    0.000    0.000 {method 'nonzero' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.002    0.002 {built-in method builtins.exec}
        1    0.000    0.000    0.002    0.002 3277688792.py:2(solve_kleene)
        1    0.000    0.000    0.000    0.000 graph.py:759(number_of_nodes)
        1    0.000    0.000    0.002    0.002 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.len}

## Big tests

In [ ]:

V = 1000
# graph = nx.random_k_out_graph(V, 3, 3)
# graph = nx.binomial_tree(5)
graph = nx.soft_random_geometric_graph(V, .1)
init_from_nx(graph)
drawnext(base_only=True)


In [ ]:
drawnext(filter_c=11)
shell_example(.1)


## Open Stored datasets

In [10]:

def open_data_set(path: str = REAL_PATH, filter=None):
    if isinstance(filter, int):
        filter = (filter,)
    G = nx.DiGraph()
    with open(path, "r") as f:
        for i, line in enumerate(f.readlines()):
            words = line.split(' ')
            if len(words) == 1: 
                print(words[0])
                continue
            s, l, t, _ = words
            if filter is None or int(l) in filter:
                G.add_edge(int(s),int(t), label=int(l))
    print((G.number_of_nodes(), G.number_of_edges()))
    return G

def open_data_set_scan_subgraphs(path: str = REAL_PATH, max_label=36):
    for subgraph in range(max_label+1):
        G = nx.DiGraph()
        with open(path, "r") as f:
            for i, line in enumerate(f.readlines()):
                words = line.split(' ')
                if len(words) == 1: 
                    continue
                s, l, t, _ = words
                if int(l) == subgraph:
                    G.add_edge(int(words[0]),int(words[2]))
        print(subgraph, (G.number_of_nodes(), G.number_of_edges()))

In [ ]:
# use pickled positions:
import pickle

# export calculated positions:
# with open('real_positions.pickle', 'wb') as f:
#     # Pickle the 'data' dictionary using the highest protocol available.
#     pickle.dump(pos_real, f, pickle.HIGHEST_PROTOCOL)

# import:

with open('real_positions.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pos_real = pickle.load(f)
plt.clf()
nx.draw(G, pos_real, node_size=20)


## TC on real data

In [11]:
# solve TC on union of label 2 and 3
G = open_data_set(SYN_PATH, filter=(1,2,3,4))
init_from_nx(G)
# drawnext(base_only=True)

9726,22158,4

(4948, 14900)


In [12]:
solve_kleene(1)
drawnext(filter_c=0, color='green')

  0%|          | 0/4948 [00:00<?, ?it/s]

In [26]:
drawnext(filter_c=0, color='green')


In [24]:
explore(0,0)

In [19]:
(N[1] & C[1]).nonzero()

(array([1], dtype=int64),)